# Code template for hand-in on reconstruction attacks.

Advanced Topics in Machine Learning, U. Copenhagen, fall 2024

Created by Rasmus Pagh with minor edits by Thomas Christensen

Queries on a hidden dataset x from {-1,+1}^100 can be made via the query method below
which calls a web API that allows dot product queries with vectors in {-1,+1}^100.
To protect data, Laplace noise is added to responses. Using the techniques you have
seen in the lecture it is possible to partially reconstruct the dataset using 200 queries.
To make sure that you get a unique challenge, choose any unique string as your challenge
identifier. The web API will keep track of the number of queries made for each identifier.

# Support function for querying the web API

In [1]:
import numpy as np
import requests as rq

# Retrieve answer to challenge for a given query
def query(challenge_id, query_vector, submit=False):
    # Only alphanumeric challenge_id and vextor entries in {-1,+1} are allowed:
    assert(challenge_id.isalnum())
    assert(np.max(np.minimum(np.abs(query_vector-1),np.abs(query_vector+1)))==0)

    # if query array is 1d, make it 2d
    if query_vector.ndim == 1:
        query_vector = query_vector.reshape(1,-1)

    payload = { 'challengeid': challenge_id, 'submit': submit,
                'query': str(query_vector.tolist()) }
    response = rq.post("https://baconbreaker.pythonanywhere.com/query", data = payload).json()
    if submit == False:
        return np.array(eval(response['result']))
    else:
        return response['result']

# Making random queries to the API

In [15]:
challenge_id = 'QWERTY05' # identifier for hidden dataset
n = 100 # number of entries in hidden dataset
num_queries = 2*n # number of queries to be asked

queries = np.random.choice([-1,+1], size=(num_queries,n)) # Set of random queries
query_results = query(challenge_id, queries)

print(query_results)

[-10.   9.  -5.  -3.  19.  11. -15.  18.   3. -31.   1.   5.  15.  10.
  11.   8.  19.  84.  57. -27. -38.   6.   2.  26.  36. -10.   0. -12.
 -31.  26. -14.  -3.  -2. -27.  19.   2. -10.  29. -28.  -8. -15.  -3.
  23. -22.  15.  29.  -6.   4.  -8.   5. -12.  -9.   9.   5. -14.  15.
 -11.   5.   2.   1.  -7. -10.  16.  22.   8.   4.   5.   1.   1.  -0.
  -7.   1.  19. -22.  -3.   9.  -8.  20.  -1.   3. -11.  -8.  -3.  -4.
 -33.  -8. -20. -11.   7.   3.   3.  34.   5.   2.  -4.   9.   1.  -7.
  10.  -2. -27.  29. -13.   6. -22.   8.   6.  30.  -2. -29. -10.   7.
 -33.   3.   5.   3. -19.   1.   2.   4.   8.  14.   8.  12.  33.  -8.
   2.   6.  -1.   9.   4.   4. -12.  21.   8.   6. -24. -17.   6.  20.
  -6.  -7. -20.   3.  -2.  -8.   4.  10.   2. -12.   4. -30.  15. -27.
   1.  10. -17.  39.  13.  -3.  15. -20.   4.  -4. -13.  -6.   5.  14.
  -6. -23.  -5. -18. -18. -19. -18.  28. -33.  13.  21.  -3.  -7.  10.
   0.  10. -29.  15. -18.  13.  11. -24.   2. -12.  -5. -13. -14.  -2.
  -1. 

# Baseline reconstruction attack: Guess equals the best query

Copy the notebook and replace with your attack. The first submission for a given challenge_id is stored on the server and on the leaderboard: http://baconbreaker.pythonanywhere.com/leaderboard/?n=100

In [3]:
best_query_number = np.argmax(query_results)
best_query = queries[best_query_number]
best_query_result = query(challenge_id, best_query, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")


Reconstruction attack achieves fraction 0.58 correct values


In [16]:
import numpy as np
from scipy.optimize import linprog

Q = queries
y = query_results 

m, n = Q.shape
c = np.hstack([np.zeros(n), np.ones(m)]) 

A_ub = np.vstack([np.hstack([Q, -np.eye(m)]), np.hstack([-Q, -np.eye(m)])])
b_ub = np.hstack([y, -y])

bounds = [(-1, 1)] * n + [(0, None)] * m  

res = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, method='highs')

x = res.x[:n]
res = np.where(x > 0, 1, -1)
print("x:", res)

best_query_result = query(challenge_id, res, submit=True)
print(f"\nReconstruction attack achieves fraction {(1 + best_query_result / n) / 2} correct values")

x: [ 1  1  1 -1 -1  1  1 -1  1  1  1  1 -1 -1 -1  1  1  1 -1 -1 -1  1 -1 -1
 -1 -1  1 -1 -1  1  1 -1 -1  1  1 -1 -1 -1 -1 -1  1  1 -1 -1 -1 -1 -1 -1
  1 -1  1  1 -1 -1 -1 -1 -1  1 -1  1  1 -1 -1  1 -1  1 -1 -1  1  1 -1 -1
  1 -1 -1  1  1 -1 -1 -1 -1 -1 -1  1  1  1  1  1 -1 -1 -1  1 -1  1 -1  1
 -1 -1 -1 -1]

Reconstruction attack achieves fraction 0.8 correct values


In [97]:
best_query_result

array([68.])